## Подготовка

### Импорт всех необходимых библиотек

In [2]:
#загружаем библиотеки для работы с данными
import pandas as pd
import numpy as np

#загружаем библиотеку для отображения времени выполнения ячейки и времени выполнения кода
import time

#загружаем библиотеку для корректной загрузки датасетов
import os

#загружаем библиотеку для проверки корректности url ссылки
import requests

#загружаем библиотеки для управления уведомлениями
import logging
import warnings

#загружаем классы для визуализации
from matplotlib import pyplot as plt
import seaborn as sns

#загружаем класс для расчета TF-IDF и мешка слов
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

#загрузка модуля и словарей для лематизации текста на английском языке
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')

#загружаем модули и словарь для добавления POS-тегов
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag

#загрузка библиотеки модуля для обработки стоп-слов и архива со стоп-словами 
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords')

#загрузка библиотеки для создания регулярных выражений
import re

# загружаем классы для подготовки данных
import sklearn
from sklearn.model_selection import train_test_split

#загружаем модули для создания piepline
from imblearn.pipeline import Pipeline

# загружаем нужные модели
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from catboost import CatBoostClassifier

#загружаем инструменты для подбора гиперпараметров
from sklearn.model_selection import GridSearchCV
from optuna.integration import OptunaSearchCV
from optuna import distributions
import optuna

# загружаем функции для работы с метриками
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer

#загружаем класс для устранения дисбаланса посредством андерсемплинга
from imblearn.under_sampling import RandomUnderSampler

#загружаем библиотеки для работы с моделью типа BERT
import transformers
import torch

#загружаем библиотеку для отображения прогресса при делении задачи на батчи
from tqdm import tqdm

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dshap\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dshap\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\dshap\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dshap\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Объявляем также константы

In [3]:
RANDOM_STATE = 42
TEST_SIZE = 0.25

### Загрузка и изучение данных

Загружаем файл датасета, создаем датафрейм, выводим первые 15 строк и основную информацию для ознакомления.

In [4]:
#проверяем существование указанной дирректории и в случае возврата True загружаем датасет в переменную, указав верные разделители
pth1 = '/datasets/toxic_comments.csv'
pth2 = 'https://...../toxic_comments.csv' #полная ссылка скрыта ввиду NDA
pth3 = 'toxic_comments.csv'
if os.path.exists(pth1):
    data_main = pd.read_csv(pth1, sep=',', decimal = '.', index_col = [0])
#добавляем проверку корректности url ссылки при помощи requests.get(url) и проверки status_code == 200
elif requests.get(pth2).status_code == 200:
    data_main = pd.read_csv(pth2, sep=',', decimal = '.', index_col = [0])
if os.path.exists(pth3):
    data_main = pd.read_csv(pth3, sep=',', decimal = '.', index_col = [0])
else:
    print('Something is wrong')

#выводим первые 15 строк и основную информацию датафрейма    
display(data_main.head(15))
data_main.info()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


<class 'pandas.core.frame.DataFrame'>
Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


**Созданный датафрейм:**
- имеет размерность 159292 строки
- пропущены порядковые индексы строк
- не имеет явных пропусков
- столбцы имеют корректные типы данных
- названия столбцов приведены к змеиному "регистру"
- при первичном ознакомлении неявные дубликаты не выделяются

###  Предобработка данных

Так как датасет состоит всего из двух столбцов, то выделять основные разделы по предобработке данных не будем, а основное опишем в одном небольшом пункте.

Еще раз проверим отсутствие явных пропусков

In [5]:
data_main.isna().sum().sort_values(ascending = False)

text     0
toxic    0
dtype: int64

Проверим датасет на наличие явных дубликатов

In [6]:
data_main.duplicated().sum()

0

Так как в некоторых строках есть пропущенные индексы, то обновим все индексы в датасете

In [7]:
data_main = data_main.reset_index(drop = True)

Посмотрим на распределение целевого признака

In [8]:
data_main['toxic'].value_counts()

toxic
0    143106
1     16186
Name: count, dtype: int64

*Токсичных комментариев чуть больше 10% от общего количества, явный дисбаланс, при разбиении на выборки добавим параметр `stratify`*

**На этапе предобработки данных:**
- проверено, что в датасете отсутствуют явные пропуски
- проверено наличие явных дубликатов
- из-за пропущенных индексов были обновлены индексы всего датасета
- выявлен сильный дисбаланс классов в целевом признаке

## Обучение

В нашем исследовании попробуем получить прогнозные значения целевого признака при помощи нескольких подходов:
- при ручной токенизации, лематизации, фильтрации от стоп-слов, создании частотных признаков TF-IDF, признаков на основе мешка слов и N-грамм, дальнешего обучения на моделях логистической регрессии, дерева решений и CatBoostClassisfier(). Так как модели будут обучаться долго на таком наборе данных, особенно долго обучается модель CatBoostClassisfier(), то в качестве базовой модели возьмем  LogisticRegression(), для модели DecisionTreeClassifier() для оптимизации подбора лучших гиперпараметров будем использовать OptunaSearchCV(), а у модели CatBoostClassifier() возьмем стандартный набор гиперпараметров(). Также попробуем улучшить метрику, применив андерсемплинг тренировочной выборки для устранения дисбаланса классов.
- второй подход будет заключаться в использовании предобученной модели BERT (токенизатора и модели для получения эмбеддингов), обработки текста при помощи этих моделей при помощи GPU  для ускорения расчетов и дальнейшей классификации при помощи тех же моделей LogisticRegression() и CatBoostClassisfier().
- основная метрика качества - F1

### Мешок слов, TF-IDF и N-граммы

#### Очистка и лематизация

Для лематизации строк текста будем использовать библиотеку `WordNetLemmatizer()`, входящую в состав библиотеки `NLTK` и работающей с текстом на английском языке. Для обработки всех строк текста напишем функцию для лематизации текста, включающую добавление POS-тегов, показывающих принадлежность каждого слова к определенной части речи. Также напишем функцию для очистки предложений от ненужных символов, пробелов и переведения всего текста в нижний регистр. Применим эти две функции ко всему столбцу `text` с отображением прогресса через `progress_apply`.

In [10]:
def clear_text(text):
    """ Функция очистки текста: удаление всех лишних символов, удаление лишних пробелов и перевод в нижний регистр"""
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    text = " ".join(text.split())
    text = text.lower()
    return text

def pos_tagger(nltk_tag):
    """Функция для определения основных POS-тегов"""
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

# инициилизируем объект для лемматизации
lemmatizer = WordNetLemmatizer()


def lemmatize(sentence):
    """Функция для токенизации строки текста, нахождения POS-тег для каждого токена и объединения в понятный формат для WordNet"""
    #берем строку текста, токенизируем и выделяем POS-тег для каждого токена
    pos_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))

 
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #если доступного тега нет, добавляем токен как есть.
            lemmatized_sentence.append(word)
        else:        
            #или добавляем тег для лемматизации токена
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    #объединяем лемматизированную строку текста с POS-тегами или без них
    lemmatized_sentence = " ".join(lemmatized_sentence)

    return(lemmatized_sentence)

Добавим функцию, которая позволит добавлять индикаторы прогресса к операциям pandas()

In [11]:
#применяем возможность добавления индикаторов прогресса к pandas
tqdm.pandas()

Применим функцию для очистки текста: удаление лишних символов, лишних пробелов и приведение всех слов к нижнему регистру

In [12]:
data_main['lemm_text']=data_main['text'].progress_apply(clear_text)

100%|███████████████████████████████████████████████████████████████████████| 159292/159292 [00:04<00:00, 39336.43it/s]


Применим функцию для лемматизации всего корпуса очищенного текста

In [13]:
data_main['lemm_text']=data_main['lemm_text'].progress_apply(lemmatize)

100%|█████████████████████████████████████████████████████████████████████████| 159292/159292 [16:28<00:00, 161.13it/s]


Проверим получившиеся значения

In [14]:
data_main.head(5)

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not try to edit war it s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


**Весь корпус текста лемматизирован, очищен от лишних символов и стоп-слов**

#### Деление датасета на выборки

Делим датасет на тренировочную и тестовую выборки в пропорции 75 на 25. Для сохранения пропорций разных групп в целевом признаке укажем параметр `stratify`.

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
        data_main['lemm_text'],
        data_main['toxic'],
        test_size = TEST_SIZE, 
        random_state = RANDOM_STATE,
        stratify=data_main['toxic'])

**Созданы тренировочная и тестовая выборки с лемматизированным текстом, очищенным от лишних символов и стоп-слов**

#### Создание признаков при помощи TF-IDF

Следующим шагом нам необходимо создать счетчик для вычисления TF-IDF, передав ему множество стоп-слов на английском языке. Для того, чтобы в модели не были учтены частоты слов из тестовой выборки, то `fit_transform` применим к обучающей выборке, а `transform` к тестовой. 

In [16]:
#создадим множество стоп-слов
stopwords = list(set(nltk_stopwords.words('english')))

#создадим счётчик, указав в нём стоп-слова:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

Теперь посчитаем частоты TF-IDF для тренировочной и тестовой выборок

In [17]:
X_train_tf_idf = count_tf_idf.fit_transform(X_train)
X_test_tf_idf = count_tf_idf.transform(X_test)

Посмотрим размерности полученных данных

In [18]:
print(X_train_tf_idf.shape)
print(X_test_tf_idf.shape)

(119469, 131387)
(39823, 131387)


**Признаки на основе TF-IDF созданы**

#### Создание признаков при помощи мешка слов

Создадим счетчик и также передадим ему множество со стоп-словами, которые он не должен учитывать

In [19]:
count_vect = CountVectorizer(stop_words=stopwords)

Передадим счетчику наш столбец с предварительно обработанным текстом. Счетчик выделит из корпуса уникальные слова и посчитает количество их в каждой строке столбца. Отдельные буквы счетчик не учитывает.

In [20]:
X_train_bow = count_vect.fit_transform(X_train)
X_test_bow = count_vect.transform(X_test)

Посмотрим размерности полученных данных

In [21]:
print(X_train_bow.shape)
print(X_test_bow.shape)

(119469, 131387)
(39823, 131387)


**Признаки на основе мешка слов созданы**

#### Создание признаков при помощи N-грамм

Создадим счетчик для N-грамм. Стоп-слова убирать не будем, так как их удаление может привести к потере важных контекстных связей. Укажем просчет для 2 слов в N-gramm.

In [22]:
count_n_gramm = CountVectorizer(ngram_range=(2,2))

Передадим счетчику наш столбец с предварительно обработанным текстом.

In [23]:
X_train_n_gramm = count_n_gramm.fit_transform(X_train)
X_test_n_gramm = count_n_gramm.transform(X_test)

Посмотрим размерности полученных данных

In [24]:
print(X_train_bow.shape)
print(X_test_bow.shape)

(119469, 131387)
(39823, 131387)


**Признаки на основе N-gramm созданы**

Приступим к обучению моделей

#### Обучение модели LogisticRegression() на разных наборах признаков

Так как модели обучаются долго, то не будем использовать автоматизированный подбор с гиперапарметрами сразу для всех моделей в пайплайне. Будем обучать базовые модели и смотреть на изменение метрики. Сначала построим базовую модель LogisticRegression() для мешка слов, TF-IDF и N-gramm, после этого набор признаков, на котором получится наивысшая метрика при кросс-валидации на логистической регрессии используем для модели дерева решений и подберем гиперпараметры при помощи OptunaSearchCV() с небольшим числом итераций, посмотрев как отработает вообще эта модель при таком огромном количестве признаков, после этого обучим CatBoostClassifier() на стандартном наборе гиперпараметров.

Обучим нашу первую базовую модель логистической регресии на мешке слов, помимо основной метрики глянем метрику ROC-AUC, чтобы посмотреть на качество работы модели в общем . Для отображения одновременно нескольких метрик вместо `cross_val_score` будем использовать `cross_validate`, передав ему словарь с созданными метриками через `make_scorer`, а для корректного расчета `roc_auc` укажем параметры `response_method='predict_proba'` и  `greater_is_better=True`

In [25]:
model_lr_bow = LogisticRegression(max_iter=1000)

scoring = {
    'f1': make_scorer(f1_score),
    'roc_auc': make_scorer(roc_auc_score,
                           response_method='predict_proba', 
                           greater_is_better=True)
            }

score_lr_bow = cross_validate(
    model_lr_bow,
    X_train_bow,
    y_train,
    scoring = scoring,
    n_jobs= -1)

print('Метрики при кросс-валидации у первой базовой модели LogisticRegression(), обученной на мешке слов')
print(f'F1_score: {score_lr_bow["test_f1"].mean():.4f}')
print(f'ROC-AUC: {score_lr_bow["test_roc_auc"].mean():.4f}')

Метрики при кросс-валидации у первой базовой модели LogisticRegression(), обученной на мешке слов
F1_score: 0.7526
ROC-AUC: 0.9523


Попробуем улучшить метрику F1, обучив нашу базовую модель на частотных признаках TF-IDF, для сравнения будем смотреть только основную метрику F1

In [26]:
model_lr_tf_idf = LogisticRegression(max_iter=1000)

score_lr_tf_idf = cross_val_score(
    model_lr_tf_idf,
    X_train_tf_idf,
    y_train,
    scoring = 'f1').mean()

print(f'Метрика F1_score при кросс-валидации у базовой модели LogisticRegression(), обученной на TF-IDF: {score_lr_tf_idf:.4f}')

Метрика F1_score при кросс-валидации у базовой модели LogisticRegression(), обученной на TF-IDF: 0.7171


Метрика получилась ниже, чем у модели, обученной на мешке слов.

Теперь обучим нашу базовую модель логистической регресии на N-gramm

In [27]:
model_lr_n_gramm = LogisticRegression(max_iter=1000)

score_lr_n_gramm = cross_val_score(
    model_lr_n_gramm,
    X_train_n_gramm,
    y_train,
    scoring = 'f1').mean()

print(f'Метрика F1_score при кросс-валидации у базовой модели LogisticRegression(), обученной на N-gramm: {score_lr_n_gramm:.4f}')

Метрика F1_score при кросс-валидации у базовой модели LogisticRegression(), обученной на N-gramm: 0.5948


Метрика F1 на N-gramm получилась еще более низкой.

Попробуем улучшить метрику качества, подобрав гиперпараметры к нашей лучшей модели LogisticRegression(), обученной на мешке слов

In [28]:
# словарь для модели  LogisticRegression()
param_grid = {  
             'C': range(2,15),
            'class_weight': ['balanced', None]
            }

model_lr_best = LogisticRegression(max_iter=1000, solver = 'lbfgs', penalty = 'l2')

start = time.time()

grid_search = GridSearchCV(
    model_lr_best, 
    param_grid=param_grid, 
    cv=5, 
    scoring = 'f1',
    n_jobs=-1
)
grid_search.fit(X_train_bow, y_train)

grid_search_quit_time = time.time() - start
print('Характеристики при GridSearchCV')
print(f'Время подбора гиперпараметров для моделей: {grid_search_quit_time}')


print('Лучшая модель и её параметры:\n\n', grid_search.best_estimator_)
print ('Метрика F1 для лучшей модели, полученная при кросс-валидации:', round(grid_search.best_score_,3))

Характеристики при GridSearchCV
Время подбора гиперпараметров для моделей: 305.14443254470825
Лучшая модель и её параметры:

 LogisticRegression(C=3, max_iter=1000)
Метрика F1 для лучшей модели, полученная при кросс-валидации: 0.761


**Благодаря подбору гиперпараметра `C=3` удалось немного улучшить метрику F1 модели LogisticRegression() до 0.761**

**На первом этапе обучения моделей были получены следующие результаты:**
- созданы разные наборы признаков на основе мешка слов, TF-IDF и N-gramm
- были обучены несколько базовых моделей LogisticRegression() на разных данных, метрики качества F1 получились следующие:
   - Мешок слов - 0.7522 ( при этом ROC-AUC составил 0.9523)
   - TF-IDF - 0.7171
   - N-gramm - 0.5947
- при подборе гиперпараметров модель ***LogisticRegression(C=3, class_weight=None, max_iter=1000, solver = 'lbfgs', penalty = 'l2')*** улучшила метрику F1 до 0.761. 

### Устранение дисбаланса и повторное обучение моделей

Попробуем улучшить метрику лучшей модели. Данные, которые показали наибольшую метрику - это мешок слов, поэтому сделаем андерсемплинг тренировочной выборки при помощи RandomUnderSampler(), устранив большой дисбаланс классов и обучим заново модель LogisticRegression(). Подберем гиперпараметр `C` для модели LogisticRegression() при помощи GridSearchCV(). Для того, чтобы оценивание при кросс-валидации происходило корректно, будем использовать `pipeline` из библиотеки `imblearn` и семплирование будет в нем определенным шагом.

#### Устранение дисбаланса

Составим pipeline, в котором RandomUnderSampler() будет шагом перед моделями. Гиперпараметр C подберем при помощи GridSearch() для модели LogisticRegression(). Обучать будем на наборе тренировочных данных, на которых модель LogisticRegression() показала наивысшую метрику из прошлого раздела - мешке слов.

In [30]:
#обьявим семплер
sampler = RandomUnderSampler(random_state=RANDOM_STATE)

Создадим пайплайн и получим лучший набор гиперпараметров

In [31]:
pipeline = Pipeline([
    ('sampling', RandomUnderSampler(random_state=RANDOM_STATE)),
    ('models', LogisticRegression())
])

param_grid_samp = [
    # словарь для модели LogisticRegression()
    {
        'models': [LogisticRegression(max_iter=1000, 
                                      solver = 'lbfgs', 
                                      penalty = 'l2')],
        'models__C': range(3,14),
        'models__class_weight' : [None]
            },
    #словарь для модели CatBoostClassifier()
        {
        'models': [CatBoostClassifier(verbose = 0, 
                                 iterations = 1000, 
                                 learning_rate=0.1, 
                                 eval_metric = 'F1', 
                                 random_seed = RANDOM_STATE)]
    }
]

start = time.time()

grid_samp = GridSearchCV(
    pipeline, 
    param_grid=param_grid_samp, 
    cv=5, 
    scoring= 'f1',
    n_jobs=-1
)
grid_samp.fit(X_train_bow, y_train) 

grid_search_time = time.time() - start
print('\n\nХарактеристики при GridSearchCV')
print(f'Время подбора гиперпараметров для моделей: {grid_search_time}')

print('Лучшая модель и её параметры на тренировочном датасете:\n\n', grid_samp.best_estimator_)
print('Метрика F1 для лучшей модели, полученная при кросс-валидации:\n', grid_samp.best_score_)



Характеристики при GridSearchCV
Время подбора гиперпараметров для моделей: 702.3254988193512
Лучшая модель и её параметры на тренировочном датасете:

 Pipeline(steps=[('sampling', RandomUnderSampler(random_state=42)),
                ('models',
                 <catboost.core.CatBoostClassifier object at 0x000001F61A0599D0>)])
Метрика F1 для лучшей модели, полученная при кросс-валидации:
 0.7174311291361695


Метрика намного ниже, чем у базовой модели LogisticRegression(). Взглянем на таблицу результатов для лучших моделей

In [32]:
#устанавливаем настройки, чтобы таблица с результатами отображалась полностью
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

#получаем таблицу с работой всех моделей
result_models_rate = pd.DataFrame(grid_samp.cv_results_).sort_values(by = 'rank_test_score')

#отображаем нужные столбцы и выводим для наглядности первые 5 лучших моделей
display(result_models_rate[
    ['rank_test_score', 'param_models','mean_test_score','mean_fit_time', 'mean_score_time','params']
                        ].head(10))

,rank_test_score,param_models,mean_test_score,mean_fit_time,mean_score_time,params
11,1,<catboost.core.CatBoostClassifier object at 0x000001F67D1F8A70>,0.717431,340.846026,0.950456,{'models': <catboost.core.CatBoostClassifier object at 0x000001F67D1F8A70>}
0,2,LogisticRegression(max_iter=1000),0.668357,10.467639,0.054180,"{'models': LogisticRegression(max_iter=1000), 'models__C': 3, 'models__class_weight': None}"
1,3,LogisticRegression(max_iter=1000),0.665777,11.078612,0.052358,"{'models': LogisticRegression(max_iter=1000), 'models__C': 4, 'models__class_weight': None}"
2,4,LogisticRegression(max_iter=1000),0.663194,12.895667,0.049964,"{'models': LogisticRegression(max_iter=1000), 'models__C': 5, 'models__class_weight': None}"
3,5,LogisticRegression(max_iter=1000),0.661509,12.922465,0.050746,"{'models': LogisticRegression(max_iter=1000), 'models__C': 6, 'models__class_weight': None}"
4,6,LogisticRegression(max_iter=1000),0.659240,11.287034,0.053934,"{'models': LogisticRegression(max_iter=1000), 'models__C': 7, 'models__class_weight': None}"
5,7,LogisticRegression(max_iter=1000),0.657477,12.895392,0.053604,"{'models': LogisticRegression(max_iter=1000), 'models__C': 8, 'models__class_weight': None}"
6,8,LogisticRegression(max_iter=1000),0.656968,12.778675,0.054131,"{'models': LogisticRegression(max_iter=1000), 'models__C': 9, 'models__class_weight': None}"
7,9,LogisticRegression(max_iter=1000),0.655345,12.443288,0.049402,"{'models': LogisticRegression(max_iter=1000), 'models__C': 10, 'models__class_weight': None}"
8,10,LogisticRegression(max_iter=1000),0.654512,11.293372,0.064078,"{'models': LogisticRegression(max_iter=1000), 'models__C': 11, 'models__class_weight': None}"


***Андерсемплинг тренировочной выборки не принес повышение метрики F1***

В качестве эксперимента обучим модель DecisionTreeClassifier() и подберем гиперпараметры при помощи OptunaSearchCV(), кол-во итераций поставим небольшое - 120, чтобы не ждать слишком долго. Так как андерсемплинг не принес улучшение метрик у других моделей, а Optuna не работает в pipeline, чтобы метрика при кросс-валидации считалась корректно, будем обучать модель на всей тренировочной выборке на мешке слов, так как у базовой модели был получен наилучший результат метрики на этом наборе данных.

In [49]:
#отключение уведомлений от Optuna о ходе выполнения работы, оставляем только уведомления о критических ошибках
warnings.filterwarnings("ignore")
logging.getLogger("optuna").setLevel(logging.ERROR)

# запускаем таймер
start = time.time()

parameters = {
    'max_depth': distributions.IntDistribution(2, 16),
    'min_samples_split': distributions.IntDistribution(2,12),
    'min_samples_leaf': distributions.IntDistribution(1, 12),
    'max_features': distributions.IntDistribution(1, 12)
}

oscv = OptunaSearchCV(
        DecisionTreeClassifier(random_state=RANDOM_STATE),
        parameters,
        cv=5,
        scoring='f1',
        n_trials = 120,
        random_state=RANDOM_STATE
)

# поиск гиперпараметров
oscv = oscv.fit(X_train_bow,y_train)


# считаем, сколько секунд прошло с начала запуска
oscv_search_time = time.time() - start
print(f'Время поиска:{oscv_search_time}')

#лучшие гиперпараметры
print('\nЛучший набор гиперпараметров:', oscv.best_params_)
#лучшая метрика качества
print(f'\nМетрика F1 при кросс-валидации у лучшей модели  DecisionTreeClassifier() составила {oscv.best_score_:.7f}')

Время поиска:220.06256437301636

Лучший набор гиперпараметров: {'max_depth': 13, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_features': 11}

Метрика F1 при кросс-валидации у лучшей модели  DecisionTreeClassifier() составила 0.0050847


**Из-за большого количества признаков (более 120 000) и относительно малого числа итераций метрика дерева решений составила всего 0.005**

Попробуем добавить в наш итоговый пайплайн первым этапом векторизацию выборок, чтобы избежать утечки данных при обучении и получить более корректный результат при подсчете метрики кросс-валидацией. При векторизации выборок мы обучали CountVectorizer() на тренировочной выборке, а трансформировали и тренировочную и тестовую. Если при кросс-валидации тренировочная выборка разделится на несколько фолдов (4 фолда тренировочные и 1 валидационный в нашем случае), а CountVectorizer() уже обучался на всей тренировочной выборке изначальной, то у нас получается, что при кросс-валидации созданные валидационные фолды уже частично «засвечены» словарём, который формировался с их участием, что приводит к утечке данных. Поэтому нужно в пайплайне сделать отдельный шаг по векторизации, чтобы при делении на фолды при кросс-валидации CountVectorizer() обучался только на тренировочных фолдах при делении тренировочной выборки.

In [53]:
pipeline_all = Pipeline([
    ('vect', CountVectorizer(stop_words=stopwords)),
    ('models', LogisticRegression())
])

param_grid_all = [
    # словарь для модели LogisticRegression()
    {
        'models': [LogisticRegression(max_iter=1000, 
                                      solver = 'lbfgs', 
                                      penalty = 'l2')],
        'models__C': range(3,14),
        'models__class_weight' : ['balanced','None']
            },
    #словарь для модели CatBoostClassifier()
        {
        'models': [CatBoostClassifier(verbose = 0, 
                                 iterations = 1000, 
                                 learning_rate=0.1, 
                                 eval_metric = 'F1', 
                                 random_seed = RANDOM_STATE)]
    }
]

start = time.time()

grid_all = GridSearchCV(
    pipeline_all, 
    param_grid=param_grid_all, 
    cv=5, 
    scoring= 'f1',
    n_jobs=-1
)

#используем просто очищенные данные и лемматизированные, но которые еще на векторизовались и не семплировались
grid_all.fit(X_train, y_train) 

grid_search_time = time.time() - start
print('\n\nХарактеристики при GridSearchCV')
print(f'Время подбора гиперпараметров для моделей: {grid_search_time}')

print('Лучшая модель и её параметры на тренировочном датасете:\n\n', grid_all.best_estimator_)
print('Метрика F1 для лучшей модели, полученная при кросс-валидации:\n', grid_all.best_score_)



Характеристики при GridSearchCV
Время подбора гиперпараметров для моделей: 1277.3582141399384
Лучшая модель и её параметры на тренировочном датасете:

 Pipeline(steps=[('vect',
                 CountVectorizer(stop_words=['shouldn', 'wasn', "haven't",
                                             "you'll", "hadn't", 'few', 'hadn',
                                             'isn', 'that', 'their', 'haven',
                                             'my', 'when', 'any', "won't",
                                             "shan't", 'd', "weren't",
                                             'themselves', 'because', 'yours',
                                             'ain', 'down', 'during', "you've",
                                             'shan', 'do', 'nor', 'against',
                                             'our', ...])),
                ('models',
                 LogisticRegression(C=3, class_weight='balanced',
                                    max_iter=1000))])

**Метрика F1 теперь считается более корректно и стала немного меньше, чем когда мы считали метрику и делали подбор гиперпараметров при помощи кросс-валидации, но использовали сразу подготовленные данные.**

**На первом этапе обучения моделей были получены следующие результаты:**
- данные были вручную лемматизированы, убраны лишние знаки и отфильтрованы стоп-слова
- в результате создания частотных признаков TF-IDF, мешка слов и N-gramm получилось 142384 столбца для каждого созданного набора признаков
- базовая модель LogisticRegression() показала метрику F1 при кросс-валидации на:
   - мешке слов - 0.76
   - TF-IDF - 0.71
   - N-gramm - 0.57
- при подборе гиперпараметров для улучшения метрики модель ***LogisticRegression() (C=3, class_weight=None, max_iter=1000, solver = 'lbfgs', penalty = 'l2')*** улучшила метрику F1 до 0.761 на мешке слов.
- лучшим набором признаков при работе базовой модели LogisticRegression() оказался мешок слов, поэтому для улучшения метрики в этом наборе был проведен андерсемплинг на тренировочной выборке и устранен дисбаланс классов. Также был составлен pipeline, включающий шаг по семплированию для корректной оценки метрики и заново обучена модель LogisticRegression() и модель CatBoostClassifier(). Метрика F1 на мешке слов после устранения дисбаланса оказалась ниже у всех моделей, чем при обучении на полном наборе данных.
- в качестве эксперимента также была обучена модель DecisionTreeClassifier() и подобраны гиперпараметры при помощи OptunaSearchCV(), из-за большого количества входнных признаков эта модель не смогла показать приемлимый результат и метрика F1 составила всего 0.005.
- также был составлен еще один ***пайплайн, включающий векторизацию при помощи  CountVectorizer()*** отдельным шагом для более корректной оценки метрики F1 и предотвращения утечки данных. Модель с подобранными гиперпараметрами ***LogisticRegression(C=3, class_weight='balanced',  max_iter=1000*** показала метрику F1 немного меньше, чем когда векторизация тренировочной выборки происходила до использования пайплайна, и составила 0.751. Так как это наиболее корректная оценки метрики F1, то за лучшую модель первого этапа обучения возьмем именно эту модель.



Теперь попробуем улучшить метрику и обучить модели на эмбеддингах

### Векторные представления BERT

Объявляем токенизатор BERT, предобученный на английских токсичных текстах в онлайн-сообществах.

In [56]:
#инициализация токенизатора модели Bert
tokenizer = transformers.BertTokenizer.from_pretrained('unitary/toxic-bert')

Применим токенизацию к каждой строке столбца `text`, после найдем самую длинную строку с текстом и добавим во все остальные строки 0 в конце строки для того, что бы все строки сравнялись по длине, так как модель BERT работает только так. Также создадим матрицу-маску `attention_mask`, чтобы явно указать при создании эмбеддингов какие значения модели считать неважными (все добавленные нули в конце строк для увеличения длины до самой длинной строки в корпусе текста). Добавим параметры `max_length=150`, `truncation=True`, чтобы строки обрезались до максимальной указанной нами длины.

In [58]:
#применяем токенизатор и лематизатор к каждой строке текста
tokenized = data_main['text'].apply(
    lambda x: tokenizer.encode(x, max_length=150, 
                               truncation=True, 
                               add_special_tokens=True))

#находим самую длиннную строку с текстом, чтобы во все остальные строки с текстом добавить 0 справа до одинаковой длины
#так как модель BERT сможет работать только со строками одинаковой длины
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

#в каждую токенизированную и лематизированную строку с текстом добавляем нули в конце и увеличиваем каждый массив (строку) до самой длинной строки
#благодаря np.array, в следующей команде можно использовать np.where без циклов и лямбда функций сразу одной командой
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

#создаем маску для этого корпуса текста - даем модели понять, где искусственно добавленны нули и такие слова нужно пропускать
attention_mask = np.where(padded != 0, 1, 0)

Проверяем доступность GPU для расчета эмбеддингов.

In [59]:
print('Доступность GPU:', torch.device("cuda" if torch.cuda.is_available() else "cpu"))

Доступность GPU: cuda


Объявляем предобученную модель для создания эмбеддингов и расчеты переключаем на `cuda`

In [60]:
model_bert = transformers.BertModel.from_pretrained('unitary/toxic-bert').to('cuda')

Разобьем процесс по созданию эмбеддингов на батчи по 150 строк, для того, чтобы не перегрузить компьютер слишком большим объемом, который будет занят в оперативной памяти и расчитаем эмбеддинги. Для того, чтобы модель быстрее сработала, укажем `with torch.no_grad()`, чтобы не считался градиент. При создании тензоров укажем .to('cuda'), а для дальнейшей обработки массива с эмбендингами передадим их обратно на cpu, так как gpu не умеет работать с массивами numpy. 

In [61]:
#разобъем создание всех эмбеддингов на части - батчи, чтобы хватило оперативной памяти, потом в самом конце снова соединим вместе все части
batch_size = 150

#пустой список для записи туда на каждой итерации части эмбедингов, которые получаются за один батч
embeddings = []

#тут делается 2 задачи: 1- функция tqdm просто добавляется перед range, чтобы отображать выполнение каждой итерации цикла
#2 - кол-во строк в корпусе целочисленно с округлением до нижнего целого значения делится на размер батча
for i in tqdm(range(padded.shape[0] // batch_size)):

#первые 150 строк (размер батча) превращаем в тензоры 
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).to('cuda') 

#тоже самое делаем для маски
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).to('cuda')

#с помощью этой строки ускоряем вычисление, в библиотеке torch указываем, что градиенты не нужны: модель BERT обучать не будем.        
    with torch.no_grad():

#получаем эмбеддинги для первых 150 строк, для первого батча, передавая в модель BERT готовые тензоры строк и тензоры масок
        batch_embeddings = model_bert(batch, attention_mask=attention_mask_batch)

#добавляем в наш список с эмбеддингами первые 150 эмбеддингов и передаем их обратно в gpu
    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

100%|██████████████████████████████████████████████████████████████████████████████| 1061/1061 [11:13<00:00,  1.57it/s]


Объединим все массивы в один и получим итоговые признаки

In [62]:
#объединение всех массивов в один общий
features = np.concatenate(embeddings)

Проверим размерность

In [63]:
features.shape

(159150, 768)

Разделим на тренировочную и тестовую выборки

In [64]:
#разбиение данных для обучения модели
X_train_bert, X_test_bert, y_train_bert, y_test_bert = train_test_split(
        features,
        data_main['toxic'][:159150],
        test_size = TEST_SIZE,
        random_state = RANDOM_STATE,
        stratify=data_main['toxic'][:159150])

Обучим базовую модель логистической регрессии

In [65]:
model_lr_bert = LogisticRegression(max_iter=1000)

score_lr_bert = cross_validate(
    model_lr_bert,
    X_train_bert,
    y_train_bert,
    scoring = scoring,
    n_jobs= -1)

print('Метрики при кросс-валидации у модели LogisticRegression(), обученной на эмбеддингах после андерсемплинга')
print(f'F1_score: {score_lr_bert["test_f1"].mean():.4f}')
print(f'ROC-AUC: {score_lr_bert["test_roc_auc"].mean():.4f}')

Метрики при кросс-валидации у модели LogisticRegression(), обученной на эмбеддингах после андерсемплинга
F1_score: 0.9418
ROC-AUC: 0.9971


**Метрика при кросс-валидации получилась намного выше, чем у моделей LogisticRegression() и CatBoostClassifier(), обученных на мешке слов**

Попробуем также обучить модель CatBoostClassifier() на эмбеддингах, метрику посмотрим также при кросс-валидации

In [66]:
%%time

model_cat_bert = CatBoostClassifier(learning_rate=0.1, iterations=1000, early_stopping_rounds=10, depth = 6,
    eval_metric = 'F1')

score_cat_bert = cross_validate(
    model_cat_bert,
    X_train_bert,
    y_train_bert,
    scoring = scoring,
    n_jobs= -1)

print('Метрики при кросс-валидации у модели  CatBoostClassifier(), обученной на эмбеддингах после андерсемплинга')
print(f'F1_score: {score_cat_bert["test_f1"].mean():.4f}')
print(f'ROC-AUC: {score_cat_bert["test_roc_auc"].mean():.4f}')

Метрики при кросс-валидации у модели  CatBoostClassifier(), обученной на эмбеддингах после андерсемплинга
F1_score: 0.9391
ROC-AUC: 0.9965
CPU times: total: 3.7 s
Wall time: 7min 12s


**Метрика модели CatBoostClassifier() оказалась немного хуже, чем у модели LogisticRegression().**

**На втором этапе обучения моделей на эмбеддингах были получены следующие результаты:**
- вся тренировочная выборка с необработанным текстом была токенизирована, обработана и переведена в эмбеддинги при помощи инструментов модели BERT. При этом максимальная длина строки при токенизации была ограничена до 150 в силу вычислительных мощностей и объема оперативной памяти. Расчеты производились на gpu. При повторном обучении моделей метрика F1 при кросс-валидации получилась следующей:
   - LogisticRegression(): 0.9418
   - CatBoostClassifier(): 0.9391

***Учитывая полученные результаты, в качестве лучшей модели для тестирования будем использовать модель LogisticRegression(), обученную на эмбеддингах***

### Тестирование

Проведем тестирование лучшей модели LogisticRegression()

In [67]:
model_lr_bert = LogisticRegression(max_iter=1000)

#обучаем модель на эмбеддингах
model_lr_bert.fit(X_train_bert,y_train_bert)

#делаем предсказание
y_pred = model_lr_bert.predict(X_test_bert)
score_best = f1_score(y_test_bert, y_pred)
print(f'Метрика F1-score на тестовой выборке: {round(score_best,3)}')

Метрика F1-score на тестовой выборке: 0.938


### Проверка на адекватность

Сравним показатели с моделью DummyClassifier(), обученной на всем наборе данных

Объявим модель `DummyClassifier`, обучим ее на данных и получим метрику `F1` для тестовой выборки. Так как константная модель просто присвает всем объектам метки на основе стратегии, примененной к целевому признаку, то данные для ее обучения подготоваливать нет необходимости. В параметре `strategy` укажем стратегию `uniform` для предсказания классов случайным образом

In [68]:
#инициилизируем модель DummyClassifier
dummy_model = DummyClassifier(random_state=RANDOM_STATE,strategy='uniform')
dummy_model.fit(X_train_bert, y_train_bert)

#оценка константной модели
print(f'Метрика F1-score на тестовой выборке: {round(f1_score(y_test_bert, dummy_model.predict(X_test_bert)),3)}')

Метрика F1-score на тестовой выборке: 0.172


**Модель прошла проверку на адекватность и показала значение метрики выше, чем у константной модели DummyClassifier()**

## Выводы

**В результате исследования были проделаны следующие действия и получены следующие результаты:**
    
- ***на этапе предварительной подготовки*** были установлены и загружены необходимые библиотеки, создан основной датафрейм с данными
    
- ***на этапе предобработки данных***:
  - проверено, что в датасете отсутствуют явные пропуски
  - проверено наличие явных дубликатов
  - из-за пропущенных индексов были обновлены индексы всего датасета
  - выявлен сильный дисбаланс классов в целевом признаке

- ***на первом этапе обучения моделей были получены следующие результаты:***
  - данные были вручную лемматизированы, убраны лишние знаки и отфильтрованы стоп-слова
  - в результате создания частотных признаков TF-IDF, мешка слов и N-gramm получилось 142384 столбца для каждого созданного набора признаков
  - базовая модель LogisticRegression() показала метрику F1 при кросс-валидации на:
     - мешке слов - 0.76
     - TF-IDF - 0.71
     - N-gramm - 0.57
  - при подборе гиперпараметров для улучшения метрики модель ***LogisticRegression() (C=3, class_weight=None, max_iter=1000, solver = 'lbfgs', penalty = 'l2')*** улучшила метрику F1 до 0.761 на мешке слов.
  - лучшим набором признаков при работе базовой модели LogisticRegression() оказался мешок слов, поэтому для улучшения метрики в этом наборе был проведен андерсемплинг на тренировочной выборке и устранен дисбаланс классов. Также был составлен pipeline, включающий шаг по семплированию для корректной оценки метрики и заново обучена модель LogisticRegression() и модель CatBoostClassifier(). Метрика F1 на мешке слов после устранения дисбаланса оказалась ниже у всех моделей, чем при обучении на полном наборе данных.
  - в качестве эксперимента также была обучена модель DecisionTreeClassifier() и подобраны гиперпараметры при помощи OptunaSearchCV(), из-за большого количества входнных признаков эта модель не смогла показать приемлимый результат и метрика F1 составила всего 0.005.
  - также был состален еще один ***пайплайн, включающий векторизацию при помощи  CountVectorizer()*** отдельным шагом для более корректной оценки метрики F1 и предотвращения утечки данных. Модель с подобранными гиперпараметрами ***LogisticRegression(C=3, class_weight='balanced', max_iter=1000)*** показала метрику F1 немного меньше, чем когда векторизация тренировочной выборки происходила до использования пайплайна, и составила 0.751. Так как это наиболее корректная оценки метрики F1, то за лучшую модель первого этапа обучения возьмем именно эту модель.
- ***на втором этапе обучения моделей на эмбеддингах были получены следующие результаты:***
  - вся тренировочная выборка с необработанным текстом была токенизирована, обработана и переведена в эмбеддинги при помощи инструментов модели BERT. При этом максимальная длина строки при токенизации была ограничена до 150 в силу вычислительных мощностей и объема оперативной памяти. Расчеты производились на gpu. При повторном обучении моделей метрика F1 при кросс-валидации получилась следующей:
     - LogisticRegression(): 0.9418
     - CatBoostClassifier(): 0.9391
- ***на этапе тестрирования*** метрика F1 модели LogisticRegression() составила 0.938
- ***также модель прошла проверку на адекватность и показала значение метрики выше, чем у константной модели DummyClassifier()***